In [8]:
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState

llm = init_chat_model("openai:gpt-4o-mini")


In [9]:
class State(MessagesState):
    pass


graph_builder = StateGraph(State)

In [10]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}


graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

graph.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is your name"
         }
    ]
})

{'messages': [HumanMessage(content='what is your name', additional_kwargs={}, response_metadata={}, id='8ede961c-58a4-42a1-b879-2d91743d8bd1'),
  AIMessage(content='I’m called ChatGPT. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CP1hQs4uFhEixgahR4JBJXSDJ7Xmx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2d5b4778-c4a1-4517-9666-8397315e0e0c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 13, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}